In [73]:
import math
import numpy as np
import math
# import pandas
from optparse import OptionParser
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble
from collections import defaultdict
from copy import deepcopy
from multiprocessing import Pool
from itertools import chain
import time
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from __future__ import division
% matplotlib inline

In [113]:
# Functions to extrac the documents, query and rank information
def extractFeatures(split):
    features = []
    for i in range(2, 138):
        features.append(float(split[i].split(':')[1]))
    # Convert to tuples:
    return features

def extractQueryData(split):
    # Add tuples:
    queryFeatures = split[1].split(':')[1]
    return queryFeatures

def readDataset(path):
    dictio_quid= defaultdict(list)
    features_list=[]
    rank_list=[]
    print('Reading training data from file...')
    k=0
    with open(path, 'r') as file:
        for line in file:
            # Getting features:
            split = line.split()
            features=extractFeatures(split)
            features_list.append(features)

            # Getting the query:
            query = int(extractQueryData(split))
            #print "query: "+str(query)
            # Getting rank:
            rank = int(split[0])
            rank_list.append(rank)

            # Feeding dictionary:
            dictio_quid[query].append((features, rank))
            k+=1
            #if k==10:
            #   break
    print('Number of query ID %d' %(len(features_list)))
    return np.array(features_list), np.array(rank_list), dictio_quid


In [41]:
_, tes_ranks, __= readDataset('MSLR-WEB10K/Fold1/test.txt')


In [42]:
# Getting the list of scores:
import os
list_scores=[]
for file in os.listdir("best_models/"):
    if file.endswith(".txt"):
        list_scores.append(os.path.join("best_models/", file))

In [43]:
list_scores

['best_models/test_LM_rank_score_LambdaMART1.txt',
 'best_models/test_RB_rank_score_RB_model_r100_tc10.txt',
 'best_models/test_RN_rank_score_RN_model_e20_l2_n1_lr5e-05.txt']

In [46]:
model_names=["LambdaMart","RankBoost","RankNet"]

In [86]:
def txt_to_dictio_eval(file_name):
    
    model_output_array = np.loadtxt(file_name)
    model_output_arr_rel = np.concatenate((model_output_array, np.reshape(tes_ranks, (-1, 1))), axis=1)
    
    
    model_output_dict = defaultdict(list)
    model_output_dict_or = defaultdict(list)

    for i in model_output_arr_rel:
        model_output_dict[i[0]].append((i[3], i[2]))

    for i in model_output_dict.keys():
        model_output_dict[i].sort(reverse=True)

    for i in model_output_dict.keys():
        model_output_dict_or[i] = [(x[1] ,x[0]) for x in model_output_dict[i]]
        
    return model_output_dict_or

def get_bins(dictio_eval):
    #Get the bins:
    all_tuples=[]
    for key in dictio_eval.keys():
        all_tuples.append(dictio_eval[key])
    all_tuples=list(chain.from_iterable(all_tuples))
    all_scores=[i[0] for i in all_tuples]
    all_true=[i[1] for i in all_tuples]
    number_bins=4#len(set(all_true))
    bins = np.histogram(all_scores, bins=number_bins, range=None, normed=False, weights=None)[1]
    return bins, all_true, all_scores

def assign_to_bin(list_score,bins):
    # Get bins
    #bins = np.histogram(list_score, bins=number_bins, range=None, normed=False, weights=None)[1]
    inds = np.digitize(list_score, bins,right=True)
    return inds

import sklearn
import matplotlib.pyplot as plt
import itertools


def plot_confusion_matrix(cm, classes, title='Confusion matrix',cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    
    print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

for i in list_scores[2:3]:
    plot_confusion_matrix(sklearn.metrics.confusion_matrix(all_true, predicted_scores), [0,1,2,3,4])
    plt.savefig(str(i[12:])+'.pdf')
    print(i)